In [46]:
import time

# 0. Demos

In [47]:
# pip install accelerate
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
device = torch.device("cuda")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to(device)

max_input_length = 1024

input_text = "translate English to Germany: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


<pad> Wie er er?</s>


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# 1. CtrlSciSumm testing

In [48]:
# example from BERT paper
article = """Introduction Language model pre-training has been shown tobe effective for improving many natural languageprocessing tasks (Dai and Le, 2015; Peters et al.,2018a; Radford et al., 2018; Howard and Ruder,2018). These include sentence-level tasks such asnatural language inference (Bowman et al., 2015;Williams et al., 2018) and paraphrasing (Dolanand Brockett, 2005), which aim to predict the re-lationships between sentences by analyzing themholistically, as well as token-level tasks such asnamed entity recognition and question answering,where models are required to produce ﬁne-grainedoutput at the token level (Tjong Kim Sang andDe Meulder, 2003; Rajpurkar et al., 2016). There are two existing strategies for apply-ing pre-trained language representations to down-stream tasks: feature-based and ﬁne-tuning. Thefeature-based approach, such as ELMo (Peterset al., 2018a), uses task-speciﬁc architectures thatinclude the pre-trained representations as addi-tional features. The ﬁne-tuning approach, such asthe Generative Pre-trained Transformer (OpenAIGPT) (Radford et al., 2018), introduces minimaltask-speciﬁc parameters, and is trained on thedownstream tasks by simply ﬁne-tuning all pre-trained parameters. The two approaches share thesame objective function during pre-training, wherethey use unidirectional language models to learngeneral language representations. We argue that current techniques restrict thepower of the pre-trained representations, espe-cially for the ﬁne-tuning approaches. The ma-jor limitation is that standard language models areunidirectional, and this limits the choice of archi-tectures that can be used during pre-training. Forexample, in OpenAI GPT, the authors use a left-to-right architecture, where every token can only at-tend to previous tokens in the self-attention layersof the Transformer (Vaswani et al., 2017). Such re-strictions are sub-optimal for sentence-level tasks,and could be very harmful when applying ﬁne-tuning based approaches to token-level tasks suchas question answering, where it is crucial to incor-porate context from both directions. In this paper, we improve the ﬁne-tuning basedapproaches by proposing BERT: BidirectionalEncoder Representationsfrom Transformers.BERT alleviates the previously mentioned unidi-rectionality constraint by using a “masked lan-guage model” (MLM) pre-training objective, in-spired by the Cloze task (Taylor, 1953). Themasked language model randomly masks some ofthe tokens from the input, and the objective is topredict the original vocabulary id of the masked word based only on its context. Unlike left-to-right language model pre-training, the MLM ob-jective enables the representation to fuse the leftand the right context, which allows us to pre-In addi-train a deep bidirectional Transformer.tion to the masked language model, we also usea “next sentence prediction” task that jointly pre-trains text-pair representations. The contributionsof our paper are as follows: • We demonstrate the importance of bidirectionalpre-training for language representations. Un-like Radford et al. (2018), which uses unidirec-tional language models for pre-training, BERTuses masked language models to enable pre-trained deep bidirectional representations. Thisis also in contrast to Peters et al. (2018a), whichuses a shallow concatenation of independentlytrained left-to-right and right-to-left LMs. • We show that pre-trained representations reducethe need for many heavily-engineered task-speciﬁc architectures. BERT is the ﬁrst ﬁne-tuning based representation model that achievesstate-of-the-art performance on a large suiteof sentence-level and token-level tasks, outper-forming many task-speciﬁc architectures. • BERT advances the state of the art for elevenNLP tasks. The code and pre-trained mod-els are available at https://github.com/google-research/bert."""
article

'Introduction Language model pre-training has been shown tobe effective for improving many natural languageprocessing tasks (Dai and Le, 2015; Peters et al.,2018a; Radford et al., 2018; Howard and Ruder,2018). These include sentence-level tasks such asnatural language inference (Bowman et al., 2015;Williams et al., 2018) and paraphrasing (Dolanand Brockett, 2005), which aim to predict the re-lationships between sentences by analyzing themholistically, as well as token-level tasks such asnamed entity recognition and question answering,where models are required to produce ﬁne-grainedoutput at the token level (Tjong Kim Sang andDe Meulder, 2003; Rajpurkar et al., 2016). There are two existing strategies for apply-ing pre-trained language representations to down-stream tasks: feature-based and ﬁne-tuning. Thefeature-based approach, such as ELMo (Peterset al., 2018a), uses task-speciﬁc architectures thatinclude the pre-trained representations as addi-tional features. The ﬁne-tuning approach

In [49]:
article[:1900]

'Introduction Language model pre-training has been shown tobe effective for improving many natural languageprocessing tasks (Dai and Le, 2015; Peters et al.,2018a; Radford et al., 2018; Howard and Ruder,2018). These include sentence-level tasks such asnatural language inference (Bowman et al., 2015;Williams et al., 2018) and paraphrasing (Dolanand Brockett, 2005), which aim to predict the re-lationships between sentences by analyzing themholistically, as well as token-level tasks such asnamed entity recognition and question answering,where models are required to produce ﬁne-grainedoutput at the token level (Tjong Kim Sang andDe Meulder, 2003; Rajpurkar et al., 2016). There are two existing strategies for apply-ing pre-trained language representations to down-stream tasks: feature-based and ﬁne-tuning. Thefeature-based approach, such as ELMo (Peterset al., 2018a), uses task-speciﬁc architectures thatinclude the pre-trained representations as addi-tional features. The ﬁne-tuning approach

In [50]:
prompts = [
    "Briefly summarize this paper: {text}",
    "Write a short summary for this text: {text}",
    "Generate a short summary this paper:\n{text}",
    "{text}\n\nWrite a brief summary in a sentence or less",
    "{text}\nSummarize the aforementioned text in a single phrase.",
    "{text}\nCan you generate a short summary of the above paragraph?",
    "Write a sentence based on this summary: {text}",
    "Write a sentence based on '{text}'",
    "Summarize this article:\n\n{text}",
    "{text}\n\nWrite a long summary in more than five sentences",
    "Write five sentences based on '{text}'",
    "{text}\n\nSummarize the article. Don't include names and years"
]

outputs = []

for prompt in prompts:
    input_text = prompt.replace("{text}", article[1500:3000])
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=150)
    outputs.append(tokenizer.decode(output[0], skip_special_tokens=True))

In [51]:
outputs

['We propose a new approach to pre-training a deep bidirectional Transformer.',
 'We propose a new approach to pre-training a deep bidirectional Transformer.',
 'We improve the fine-tuning based approaches by proposing BERT: BidirectionalEncoder Representationsfrom Transformers.BERT',
 'We propose a new approach to pre-training a deep bidirectional Transformer.',
 'We propose a masked language model for pre-training a deep bidirectional Transformer.',
 'We propose a new approach to pre-training a deep bidirectional Transformer.',
 'We propose a new approach to pre-training a deep bidirectional Transformer.',
 'We improve the fine-tuning based approaches by proposing BERT: BidirectionalEncoder Representationsfrom Transformers.BERT alleviates the previously mentioned unidi-rectionality constraint by using a “masked lan-guage model” (MLM) pre-training objective, in-spired by the Cloze task (Taylor, 1953). Themasked language model randomly masks some ofthe tokens from the input, and the ob

In [15]:
prompt = "Summarize this paper:\n\n"

input_text = prompt+article[:1000]+"\n\n Summary:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

We present a novel approach to fine-tuning language models for downstream tasks, based on the idea that the pre-trained representations can be used as a starting point for the task.


In [16]:
prompt = "Provide a TLDR summary for the text: "

input_text = prompt+article[:1000]+"\n\n Summary:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

We present a novel approach to fine-tuning language models for downstream tasks, based on the idea that the pre-trained representations can be used as a starting point for the task.


In [17]:
prompt = "Provide a TLDR summary for the scientific introduction text, focus on masked language model: "

input_text = prompt+article[:1000]+"\n\n Summary:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

We propose a new approach to fine-tuning language models for downstream tasks, based on the idea of masking the language model representations.


In [18]:
prompt = "Please polish the text extracted from PDF, which talks about natural language processing, try to split concatenated words: "

input_text = prompt+article[:1000]+"\n\n Polished text:"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Introduction Language model pre-training has been shown tobe effective for improving many natural languageprocessing tasks (Dai and Le, 2015; Peters et al.,2018a; Radford et al.,2018; Howard and Ruder,2018). These include sentence-level tasks such asnatural language inference (Bowman et al., 2015;Williams et al., 2018) and paraphrasing (Dolanand Brockett, 2005), which aim to predict the re-lationships between sentences by analyzing them holistically, as well as token-level tasks such asnamed entity recognition and question answering,where models are required to produce fine-grainedoutput at the token level (Tjong Kim Sang andDe Meulder, 2003; Rajpurkar et al., 2016). There are two existing strategies for apply-ing pre


Introduction Language model pre-training has been shown tobe effective for improving many natural languageprocessing tasks (Dai and Le, 2015; Peters et al.,2018a; Radford et al., 2018; Howard and Ruder,2018). These include sentence-level tasks such asnatural language inference (Bowman et al., 2015;Williams et al., 2018) and paraphrasing (Dolanand Brockett, 2005), which aim to predict the re-lationships between sentences by analyzing themholistically, as well as token-level tasks such asnamed entity recognition and question answering,where models are required to produce ﬁne-grainedoutput at the token level (Tjong Kim Sang andDe Meulder, 2003; Rajpurkar et al., 2016). There are two existing strategies for apply-ing pre-trained language representations to down-stream tasks: feature-based and ﬁne-tuning. Thefeature-based approach, such as ELMo (Peterset al., 2018a), uses task-speciﬁc architectures thatinclude the pre-trained representations as addi-tional features. The ﬁne-tuning approach, such asthe Generative Pre-trained Transformer (OpenAIGPT) (Radford et al., 2018), introduces minimaltask-speciﬁc parameters, and is trained on thedownstream tasks by simply ﬁne-tuning all pre-trained parameters. The two approaches share thesame objective function during pre-training, wherethey use unidirectional language models to learngeneral language representations. We argue that current techniques restrict thepower of the pre-trained representations, espe-cially for the ﬁne-tuning approaches. The ma-jor limitation is that standard language models areunidirectional, and this limits the choice of archi-tectures that can be used during pre-training. Forexample, in OpenAI GPT, the authors use a left-to-right architecture, where every token can only at-tend to previous tokens in the self-attention layersof the Transformer (Vaswani et al., 2017). Such re-strictions are sub-optimal for sentence-level tasks,and could be very harmful when applying ﬁne-tuning based approaches to token-level tasks suchas question answering, where it is crucial to incor-porate context from both directions. In this paper, we improve the ﬁne-tuning basedapproaches by proposing BERT: BidirectionalEncoder Representationsfrom Transformers.BERT alleviates the previously mentioned unidi-rectionality constraint by using a “masked lan-guage model” (MLM) pre-training objective, in-spired by the Cloze task (Taylor, 1953). Themasked language model randomly masks some ofthe tokens from the input, and the objective is topredict the original vocabulary id of the masked word based only on its context. Unlike left-to-right language model pre-training, the MLM ob-jective enables the representation to fuse the leftand the right context, which allows us to pre-In addi-train a deep bidirectional Transformer.tion to the masked language model, we also usea “next sentence prediction” task that jointly pre-trains text-pair representations. The contributionsof our paper are as follows: • We demonstrate the importance of bidirectionalpre-training for language representations. Un-like Radford et al. (2018), which uses unidirec-tional language models for pre-training, BERTuses masked language models to enable pre-trained deep bidirectional representations. Thisis also in contrast to Peters et al. (2018a), whichuses a shallow concatenation of independentlytrained left-to-right and right-to-left LMs. • We show that pre-trained representations reducethe need for many heavily-engineered task-speciﬁc architectures. BERT is the ﬁrst ﬁne-tuning based representation model that achievesstate-of-the-art performance on a large suiteof sentence-level and token-level tasks, outper-forming many task-speciﬁc architectures. • BERT advances the state of the art for elevenNLP tasks. The code and pre-trained mod-els are available at https://github.com/google-research/bert.'

### Summary:

ChatGPT can do well in (1) full text polishing, like split concatenated words, (2) understand natural language instructions, (3) better keyword controlled summarization?

# Evaluate on datasets

### Preparing the dataloader 

In [42]:
!pip install nltk

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [52]:
import time
import os
import json
import nltk
# Set the environment variable TOKENIZERS_PARALLELISM to true or false
# os.environ["TOKENIZERS_PARALLELISM"] = "true"  # or "false"

import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, BartModel, BartForConditionalGeneration

from datasets import load_dataset
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bert import BERTScore

# Initialize the Rouge metric from torchmetrics
rouge = ROUGEScore()
bertscore = BERTScore()

# Check if a GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the MuP dataset
_, test_dataset = load_dataset("allenai/mup", split=['train', 'validation'])
print("#Validation", len(test_dataset))

# Create a DataLoader for the test dataset
def collate_fn(batch):
    articles = ["Summarize: " + example["text"] for example in batch]
    highlights = [example["summary"] for example in batch]
    return articles, highlights

batch_size = 20
dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

ModuleNotFoundError: Stemmer and/or `rougeLsum` requires that `nltk` is installed. Use `pip install nltk`.

### T5

In [9]:
# Load the T5 model and tokenizer
# , torch_dtype=torch.float16
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-base")


def generate_summary_batch(articles):
    inputs = tokenizer(articles, return_tensors="pt", max_length=512, padding=True, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, num_beams=4, length_penalty=2.0, temperature=0.2)
    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    return summaries

# Evaluate the model with DataLoader
num_examples = 0
predictions, targets = [], []

for articles, reference_summaries in dataloader:
    # Generate summaries
    generated_summaries = generate_summary_batch(articles)

    predictions.extend(generated_summaries)
    targets.extend(reference_summaries)
    
    num_examples += len(articles)
    print("# running", num_examples, time.asctime())
    if num_examples > 999: break

# Dump the data to a JSON file
with open("./output-t5-base-dev-prompt.json", "w") as json_file:
    json.dump(predictions, json_file, indent=4)
    json.dump(targets, json_file, indent=4)
    
rouge_score = rouge(predictions, targets)

print(rouge_score)

bert_score = bertscore(predictions, targets)
bert_score = [{key: sum(bert_score[key])/len(bert_score[key])} for key in bert_score]

print(bert_score)


/home/yanxia/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# running 20 Thu Mar 30 04:59:01 2023
# running 40 Thu Mar 30 04:59:06 2023
# running 60 Thu Mar 30 04:59:12 2023
# running 80 Thu Mar 30 04:59:17 2023
# running 100 Thu Mar 30 04:59:22 2023
# running 120 Thu Mar 30 04:59:27 2023
# running 140 Thu Mar 30 04:59:32 2023
# running 160 Thu Mar 30 04:59:37 2023
# running 180 Thu Mar 30 04:59:42 2023
# running 200 Thu Mar 30 04:59:47 2023
# running 220 Thu Mar 30 04:59:52 2023
# running 240 Thu Mar 30 04:59:58 2023
# running 260 Thu Mar 30 05:00:02 2023
# running 280 Thu Mar 30 05:00:07 2023
# running 300 Thu Mar 30 05:00:12 2023
# running 320 Thu Mar 30 05:00:17 2023
# running 340 Thu Mar 30 05:00:22 2023
# running 360 Thu Mar 30 05:00:27 2023
# running 380 Thu Mar 30 05:00:32 2023
# running 400 Thu Mar 30 05:00:37 2023
# running 420 Thu Mar 30 05:00:42 2023
# running 440 Thu Mar 30 05:00:47 2023
# running 460 Thu Mar 30 05:00:52 2023
# running 480 Thu Mar 30 05:00:57 2023
# running 500 Thu Mar 30 05:01:03 2023
# running 520 Thu Mar 30 05:0

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.p │
│ y:717 in convert_to_tensors                                                               │
│                                                                                           │
│    714 │   │   │   │   │   value = [value]                                                │
│    715 │   │   │   │                                                                      │
│    716 │   │   │   │   if not is_tensor(value):                                           │
│ ❱  717 │   │   │   │   │   tensor = as_tensor(value)                                      │
│    718 │   │   │   │   │                                                                  │
│    719 │   │   │   │   │   # Removing this for now in favor of controlling the shape with │
│    720 │   │   │   │   │   # # at-least2d                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: expected sequence of length 512 at dim 1 (got 726)

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3170385/2202051250.py:38 in <module>                                       │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3170385/2202051250.py'               │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1190 in     │
│ _call_impl                                                                                │
│                                                                                           │
│   1187 │   │   # this function, and just call forward.                                    │
│   1188 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1189 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1190 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1191 │   │   # Do not call functions when jit is used                                   │
│   1192 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1193 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:236 in forward  │
│                                                                                           │
│   233 │   │   if self.full_state_update or self.full_state_update is None or self.dist_sy │
│   234 │   │   │   self._forward_cache = self._forward_full_state_update(*args, **kwargs)  │
│   235 │   │   else:                                                                       │
│ ❱ 236 │   │   │   self._forward_cache = self._forward_reduce_state_update(*args, **kwargs │
│   237 │   │                                                                               │
│   238 │   │   return self._forward_cache                                                  │
│   239                                                                                     │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:302 in          │
│ _forward_reduce_state_update                                                              │
│            

### BART 

In [6]:
# Load the BART model and tokenizer
# , torch_dtype=torch.float16
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)


def generate_summary_batch(articles):
    inputs = tokenizer(articles, return_tensors="pt", max_length=512, padding=True, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, num_beams=4, length_penalty=2.0, temperature=0.2)
    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    return summaries

# Evaluate the model with DataLoader
num_examples = 0
predictions, targets = [], []

for articles, reference_summaries in dataloader:
    # Generate summaries
    generated_summaries = generate_summary_batch(articles)

    predictions.extend(generated_summaries)
    targets.extend(reference_summaries)
    
    num_examples += len(articles)
    print("# running", num_examples, time.asctime())
    if num_examples > 999: break

# Dump the data to a JSON file
with open("./output-bart-base-dev.json", "w") as json_file:
    json.dump(predictions, json_file, indent=4)
    json.dump(targets, json_file, indent=4)
    
rouge_score = rouge(predictions, targets)

print(rouge_score)

bert_score = bertscore(predictions, targets)
bert_score = [{key: sum(bert_score[key])/len(bert_score[key])} for key in bert_score]

print(bert_score)


Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

# running 20 Thu Mar 30 03:31:36 2023
# running 40 Thu Mar 30 03:31:42 2023
# running 60 Thu Mar 30 03:31:49 2023
# running 80 Thu Mar 30 03:31:54 2023
# running 100 Thu Mar 30 03:32:00 2023
# running 120 Thu Mar 30 03:32:05 2023
# running 140 Thu Mar 30 03:32:10 2023
# running 160 Thu Mar 30 03:32:15 2023
# running 180 Thu Mar 30 03:32:20 2023
# running 200 Thu Mar 30 03:32:26 2023
# running 220 Thu Mar 30 03:32:31 2023
# running 240 Thu Mar 30 03:32:36 2023
# running 260 Thu Mar 30 03:32:42 2023
# running 280 Thu Mar 30 03:32:47 2023
# running 300 Thu Mar 30 03:32:52 2023
# running 320 Thu Mar 30 03:32:57 2023
# running 340 Thu Mar 30 03:33:04 2023
# running 360 Thu Mar 30 03:33:09 2023
# running 380 Thu Mar 30 03:33:14 2023
# running 400 Thu Mar 30 03:33:21 2023
# running 420 Thu Mar 30 03:33:27 2023
# running 440 Thu Mar 30 03:33:32 2023
# running 460 Thu Mar 30 03:33:37 2023
# running 480 Thu Mar 30 03:33:42 2023
# running 500 Thu Mar 30 03:33:47 2023
# running 520 Thu Mar 30 03:3

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.p │
│ y:717 in convert_to_tensors                                                               │
│                                                                                           │
│    714 │   │   │   │   │   value = [value]                                                │
│    715 │   │   │   │                                                                      │
│    716 │   │   │   │   if not is_tensor(value):                                           │
│ ❱  717 │   │   │   │   │   tensor = as_tensor(value)                                      │
│    718 │   │   │   │   │                                                                  │
│    719 │   │   │   │   │   # Removing this for now in favor of controlling the shape with │
│    720 │   │   │   │   │   # # at-least2d                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: expected sequence of length 512 at dim 1 (got 726)

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3170385/2405758342.py:38 in <module>                                       │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3170385/2405758342.py'               │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1190 in     │
│ _call_impl                                                                                │
│                                                                                           │
│   1187 │   │   # this function, and just call forward.                                    │
│   1188 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1189 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1190 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1191 │   │   # Do not call functions when jit is used                                   │
│   1192 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1193 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:236 in forward  │
│                                                                                           │
│   233 │   │   if self.full_state_update or self.full_state_update is None or self.dist_sy │
│   234 │   │   │   self._forward_cache = self._forward_full_state_update(*args, **kwargs)  │
│   235 │   │   else:                                                                       │
│ ❱ 236 │   │   │   self._forward_cache = self._forward_reduce_state_update(*args, **kwargs │
│   237 │   │                                                                               │
│   238 │   │   return self._forward_cache                                                  │
│   239                                                                                     │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:302 in          │
│ _forward_reduce_state_update                                                              │
│            

### Flan-T5

In [8]:
# Load the Flan-T5 model and tokenizer
# , load_in_8bit=True
# , torch_dtype=torch.float16
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to(device)
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")


def generate_summary_batch(articles):
    inputs = tokenizer(articles, return_tensors="pt", max_length=512, padding=True, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, num_beams=4, length_penalty=2.0, temperature=0.2)
    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    return summaries

# Evaluate the model with DataLoader
num_examples = 0
predictions, targets = [], []

for articles, reference_summaries in dataloader:
    # Generate summaries
    generated_summaries = generate_summary_batch(articles)

    predictions.extend(generated_summaries)
    targets.extend(reference_summaries)
    
    num_examples += len(articles)
    print("# running", num_examples, time.asctime())
    if num_examples > 999: break

# Dump the data to a JSON file
with open("./output-flan-t5-base-dev-prompt.json", "w") as json_file:
    json.dump(predictions, json_file, indent=4)
    json.dump(targets, json_file, indent=4)
    
rouge_score = rouge(predictions, targets)

print(rouge_score)

bert_score = bertscore(predictions, targets)
bert_score = [{key: sum(bert_score[key])/len(bert_score[key])} for key in bert_score]

print(bert_score)



# running 20 Thu Mar 30 04:50:58 2023
# running 40 Thu Mar 30 04:51:05 2023
# running 60 Thu Mar 30 04:51:11 2023
# running 80 Thu Mar 30 04:51:18 2023
# running 100 Thu Mar 30 04:51:24 2023
# running 120 Thu Mar 30 04:51:31 2023
# running 140 Thu Mar 30 04:51:38 2023
# running 160 Thu Mar 30 04:51:45 2023
# running 180 Thu Mar 30 04:51:51 2023
# running 200 Thu Mar 30 04:51:58 2023
# running 220 Thu Mar 30 04:52:04 2023
# running 240 Thu Mar 30 04:52:10 2023
# running 260 Thu Mar 30 04:52:16 2023
# running 280 Thu Mar 30 04:52:23 2023
# running 300 Thu Mar 30 04:52:29 2023
# running 320 Thu Mar 30 04:52:35 2023
# running 340 Thu Mar 30 04:52:42 2023
# running 360 Thu Mar 30 04:52:48 2023
# running 380 Thu Mar 30 04:52:55 2023
# running 400 Thu Mar 30 04:53:02 2023
# running 420 Thu Mar 30 04:53:08 2023
# running 440 Thu Mar 30 04:53:15 2023
# running 460 Thu Mar 30 04:53:21 2023
# running 480 Thu Mar 30 04:53:27 2023
# running 500 Thu Mar 30 04:53:34 2023
# running 520 Thu Mar 30 04:5

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.p │
│ y:717 in convert_to_tensors                                                               │
│                                                                                           │
│    714 │   │   │   │   │   value = [value]                                                │
│    715 │   │   │   │                                                                      │
│    716 │   │   │   │   if not is_tensor(value):                                           │
│ ❱  717 │   │   │   │   │   tensor = as_tensor(value)                                      │
│    718 │   │   │   │   │                                                                  │
│    719 │   │   │   │   │   # Removing this for now in favor of controlling the shape with │
│    720 │   │   │   │   │   # # at-least2d                                                 │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: expected sequence of length 512 at dim 1 (got 726)

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3170385/2955627296.py:39 in <module>                                       │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3170385/2955627296.py'               │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1190 in     │
│ _call_impl                                                                                │
│                                                                                           │
│   1187 │   │   # this function, and just call forward.                                    │
│   1188 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1189 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1190 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1191 │   │   # Do not call functions when jit is used                                   │
│   1192 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1193 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:236 in forward  │
│                                                                                           │
│   233 │   │   if self.full_state_update or self.full_state_update is None or self.dist_sy │
│   234 │   │   │   self._forward_cache = self._forward_full_state_update(*args, **kwargs)  │
│   235 │   │   else:                                                                       │
│ ❱ 236 │   │   │   self._forward_cache = self._forward_reduce_state_update(*args, **kwargs │
│   237 │   │                                                                               │
│   238 │   │   return self._forward_cache                                                  │
│   239                                                                                     │
│                                                                                           │
│ /home/yanxia/anaconda3/lib/python3.9/site-packages/torchmetrics/metric.py:302 in          │
│ _forward_reduce_state_update                                                              │
│            